In [28]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [29]:
## load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";",)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [30]:
## split the data into training validation and test sets
train,test = train_test_split(data,test_size=0.25,random_state=50)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4271,6.9,0.420,0.20,15.4,0.043,57.0,201.0,0.99848,3.08,0.54,9.4,5
3545,7.3,0.200,0.29,19.5,0.039,69.0,237.0,1.00037,3.10,0.48,9.2,6
1079,6.3,0.340,0.29,6.2,0.046,29.0,227.0,0.99520,3.29,0.53,10.1,6
4578,4.8,0.130,0.32,1.2,0.042,40.0,98.0,0.98980,3.42,0.64,11.8,7
1702,6.5,0.270,0.19,4.2,0.046,6.0,114.0,0.99550,3.25,0.35,8.6,4
...,...,...,...,...,...,...,...,...,...,...,...,...
3330,6.7,0.230,0.33,8.1,0.048,45.0,176.0,0.99472,3.11,0.52,10.1,6
70,6.2,0.270,0.43,7.8,0.056,48.0,244.0,0.99560,3.10,0.51,9.0,6
132,6.5,0.425,0.40,13.1,0.038,59.0,241.0,0.99790,3.23,0.57,9.0,5
2014,9.0,0.550,0.30,8.1,0.026,14.0,71.0,0.99300,2.94,0.36,11.8,5


In [31]:
train_x= train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()

#test data sets
test_x = test.drop(['quality'],axis=1).values
test_y = test[['quality']].values.ravel()


## spliting the training data into train and validation sets

train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=50)


sigunature= infer_signature(train_x,train_y)


In [32]:
def train_model(parmas, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    ## def the model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    
    model = keras.Sequential(
        [
            keras.Input(shape=(train_x.shape[1],)),  # Fixed input shape to number of features
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )
    
    ## compile the model 
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=parmas["lr"], momentum=parmas["momentum"]
    ), loss="mean_squared_error", metrics=[keras.metrics.RootMeanSquaredError()])
    
    ## Train the model with lr and momentum parameters
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                  epochs=epochs, batch_size=64)
        
        ## evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]
        
        ## log the parameters
        mlflow.log_params(parmas)
        mlflow.log_metrics({'eval_rmse': eval_rmse})
        
        # log the model
        mlflow.tensorflow.log_model(model, "model", signature=sigunature)
        
        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [33]:
def objective(params):
    result = train_model(params,
                         epochs=3,
                         train_x=train_x,
                         train_y=train_y,
                         valid_x=valid_x,
                         valid_y=valid_y,
                         test_x=test_x,
                         test_y=test_y,)
    
    return result

In [37]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),  # learning rate search space
    "momentum": hp.uniform("momentum", 0.0, 1.0)  # momentum now in valid range [0, 1]
}


In [39]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    
    #conduct the hyperparameter search using the hyperopt
    
    trials=Trials()
    best= fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials,
    )
    
    best_run = sorted(trials.results, key=lambda x:x["loss"])[0]
        # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metrics({"eval_rmse": best_run["loss"]})  # Fix here
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=sigunature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval_rmse: {best_run['loss']}")

     

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 340ms/step - loss: 36.2075 - root_mean_squared_error: 6.0173
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 36.4980 - root_mean_squared_error: 6.0412 - val_loss: 34.5927 - val_root_mean_squared_error: 5.8816

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 36.9283 - root_mean_squared_error: 6.0769
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.6984 - root_mean_squared_error: 5.8904 - val_loss: 32.9932 - val_root_mean_squared_error: 5.7440

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 34.4748 - root_mean_squared_error: 5.8715
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33.2014 - root_mean_squared_error: 5.7619 - val_loss: 31.4746 - val_root_mean_squared_error: 5.6102

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 33.4663 - root_mean_squared_error: 5.7850
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s